In [59]:
import os
import json

## class명 수정
개별 한자 -> 'hanja'

In [ ]:
path = "/home/piai/Documents/val_lb/"
file_list = os.listdir(path)

print ("file_list: {}".format(file_list))

In [46]:
# json 파일 내 한자 = hanja로 통일

# CenterNet에서 max_objs 확인을 위해
_max = 0
max_file = None

# max_objs 및 개별 한자 -> 'hanja'로 통일
for file_name in file_list:
    with open(path+file_name, 'r') as f:

        json_data = json.load(f)

    _sum = 0
    for i in range(len(json_data['Text_Coord'])):
        for j in range(len(json_data['Text_Coord'][i])):
            json_data['Text_Coord'][i][j][1] = 'hanja'
            _sum += 1
            
    if _max < _sum:
        max_file = file_name
        _max = _sum
            
    with open(path+file_name, 'w') as outfile:
        json.dump(json_data, outfile)

In [47]:
print(_max, max_file)

438 KSAC_I_A06000076_009_042.json


## Dataset, Yolo 사이즈에 맞춰 수정

In [ ]:
path = "/home/piai/Documents/asd/val_img/"
path2 = "/home/piai/Documents/val_lb/"

file_list = os.listdir(path)

print ("file_list: {}".format(file_list))

In [1]:
from PIL import Image

# x center, y center 계산
def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = round(x*dw,6)
    w = round(w*dw,6)
    y = round(y*dh,6)
    h = round(h*dh,6)
    return (x,y,w,h)

In [4]:
# json 파일 txt 파일로 변환
no_match = []
for file_name in file_list:
    check_tf = False
    
    # json 불러오기
    with open(path2+file_name[:-3]+'json', 'r') as f:

        json_data = json.load(f)
            
    # json 내 개별 한자는 hanja로 변환        
    a = []
    for i in range(len(json_data['Text_Coord'])):
            for j in range(len(json_data['Text_Coord'][i])):
                a.append(json_data['Text_Coord'][i][j][0][:4])

                
    # 이미지에 대응하는 라벨링 json 파일을 Yolo 형식에 맞춰 txt로 저장
    b = ''
    image = Image.open(path+file_name)
    for aa in a:
        b += '0 '
    
        width = int(image.size[0])
        height = int(image.size[1])

        #바운딩박스 위치 가져오기
        xmin = aa[0]
        ymin = aa[1]
        xmax = aa[0]+aa[2]
        ymax = aa[1]+aa[3]

        #바운딩박스 위치 변환하기
        bbox_bf = (float(xmin), float(xmax), float(ymin), float(ymax))
        bbox = convert((width, height), bbox_bf)
        
        if max(bbox) > 1:
            check_tf = True
            break
            
        aa = list(bbox)

        aa = map(str, aa)
        b += ' '.join(aa)
        b += '\n'
       
    if check_tf:
        no_match.append(file_name)
        continue
        
    file = open('/home/piai/Documents/asd/val_hanja/%s.txt'%file_name[:-4], 'w')    # hello.txt 파일을 쓰기 모드(w)로 열기. 파일 객체 반환
    file.write(b)                     # 파일에 문자열 저장
    file.close()                     # 파일 객체 닫기

#### 이미지와 라벨링이 다른 경우 제거

In [43]:
no_match

['KSAC_I_A07000010_007_123.jpg',
 'KSAC_I_A08000053_009_179.jpg',
 'OKSAC_I_A01000027_003_040.jpg',
 'KSAC_I_A06000076_011_162.jpg',
 'KSAC_I_A06000076_020_141.jpg']

In [52]:
for nm in no_match:
    if os.path.isfile('/home/piai/yolov5/data/images/train_hanja/'+nm):
        print('y')
        os.remove('/home/piai/yolov5/data/images/train_hanja/'+nm)

## 이미지 경로 저장

In [57]:
# 이미지 경로 확인 및 저장된 이미지 이름 모두 불러오기
path = "/home/piai/yolov5/data/images/val_hanja/"
file_list = os.listdir(path)
len(file_list)

2273

In [58]:
# 각 이미지 경로를 하나로 통합
file_txt = ''
for f in file_list:
    b = path + f + '\n'
    file_txt += b

In [ ]:
# 통합된 이미지 경로를 txt 파일로 저장
file = open('/home/piai/CenterNet/convert/tutorial/val.txt', 'w')    # hello.txt 파일을 쓰기 모드(w)로 열기. 파일 객체 반환
file.write(file_txt)                     # 파일에 문자열 저장
file.close()      